In [2]:
import numpy as np
import pandas as pd 
import torch
import torch.nn as nn
import cv2
import os
import time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import wget

In [3]:
weightsfile = 'coco_data/yolov3.weights'
classfile = 'coco_data/coco.names'
cfgfile = 'coco_data/yolov3.cfg'
sample_img1 = 'coco_data/input/img.jpg'
input_dir = 'coco_data/input'
output_dir = 'coco_data/output'
nms_thesh = 0.3

In [4]:
import requests

# Download yolov3.cfg
url1 = 'https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg'
response1 = requests.get(url1)
with open('yolov3.cfg', 'wb') as file:
    file.write(response1.content)

# Download dog-cycle-car.png
url2 = 'https://github.com/ayooshkathuria/pytorch-yolo-v3/raw/master/dog-cycle-car.png'
response2 = requests.get(url2)
with open('dog-cycle-car.png', 'wb') as file:
    file.write(response2.content)

In [5]:
def parse_cfg(config_file):
    file = open(config_file,'r')
    file = file.read().split('\n')
    file =  [line for line in file if len(line)>0 and line[0] != '#']
    file = [line.lstrip().rstrip() for line in file]

    final_list = []
    element_dict = {}
    for line in file:

        if line[0] == '[':
            if len(element_dict) != 0:     # appending the dict stored on previous iteration
                    final_list.append(element_dict)
                    element_dict = {} # again emtying dict
            element_dict['type'] = ''.join([i for i in line if i != '[' and i != ']'])
            
        else:
            val = line.split('=')
            element_dict[val[0].rstrip()] = val[1].lstrip()  #removing spaces on left and right side
        
    final_list.append(element_dict) # appending the values stored for last set
    return final_list

In [6]:
class DummyLayer(nn.Module):
    def __init__(self):
        super(DummyLayer, self).__init__()
        

        
class DetectionLayer(nn.Module):
    def __init__(self, anchors):
        super(DetectionLayer, self).__init__()
        self.anchors = anchors
        
        

def create_model(blocks):
#     blocks = parse_cfg(cfgfile)
    darknet_details = blocks[0]
    channels = 3 
    output_filters = []
    modulelist = nn.ModuleList()
    
    for i,block in enumerate(blocks[1:]):
        seq = nn.Sequential()
        if (block["type"] == "convolutional"):
            activation = block["activation"]
            filters = int(block["filters"])
            kernel_size = int(block["size"])
            strides = int(block["stride"])
            use_bias= False if ("batch_normalize" in block) else True
            pad = (kernel_size - 1) // 2
            
            conv = nn.Conv2d(in_channels=channels, out_channels=filters, kernel_size=kernel_size, 
                             stride=strides, padding=pad, bias = use_bias)
            seq.add_module("conv_{0}".format(i), conv)
            
            if "batch_normalize" in block:
                bn = nn.BatchNorm2d(filters)
                seq.add_module("batch_norm_{0}".format(i), bn)

            if activation == "leaky":
                activn = nn.LeakyReLU(0.1, inplace = True)
                seq.add_module("leaky_{0}".format(i), activn)
            
        elif (block["type"] == "upsample"):
            upsample = nn.Upsample(scale_factor = 2, mode = "bilinear")
            seq.add_module("upsample_{}".format(i), upsample)
        
        elif (block["type"] == 'route'):
            # start and end is given in format (eg:-1 36 so we will find layer number from it.
            # we will find layer number in negative format
            # so that we can get the number of filters in that layer
            block['layers'] = block['layers'].split(',')
            block['layers'][0] = int(block['layers'][0])
            start = block['layers'][0]
            if len(block['layers']) == 1:               
                filters = output_filters[i + start]
                       
            
            elif len(block['layers']) > 1:
                block['layers'][1] = int(block['layers'][1]) - i 
                end = block['layers'][1]
                filters = output_filters[i + start] + output_filters[i + end]
                  
            
            route = DummyLayer()
            seq.add_module("route_{0}".format(i),route)
                
      
        elif block["type"] == "shortcut":
            from_ = int(block["from"])
            shortcut = DummyLayer()
            seq.add_module("shortcut_{0}".format(i),shortcut)
            
            
        elif block["type"] == "yolo":
            mask = block["mask"].split(",")
            mask = [int(m) for m in mask]
            anchors = block["anchors"].split(",")
            anchors = [(int(anchors[i]), int(anchors[i + 1])) for i in range(0, len(anchors), 2)]
            anchors = [anchors[i] for i in mask]
            block["anchors"] = anchors
            
            detectorLayer = DetectionLayer(anchors)
            seq.add_module("Detection_{0}".format(i),detectorLayer)
                
        modulelist.append(seq)
        output_filters.append(filters)  
        channels = filters
    
    return darknet_details, modulelist